In [1]:
import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
sys.path.append( '/home/cbisot/pycode/MscThesis/amftrack/pipeline/functions')
%matplotlib widget
from amftrack.notebooks.analysis.util import *
from amftrack.pipeline.paths.directory import path_code, directory_scratch, directory_project
from amftrack.notebooks.analysis.data_info import *
import pandas as pd
from scipy.ndimage import gaussian_filter
from scipy import ndimage
import matplotlib.pyplot as plt

from IPython.display import clear_output
import cv2
import imageio

In [3]:
def get_density_maps(exp,t,compress,kern_sizes):
    skeletons = [sparse.csr_matrix(skel) for skel in exp.skeletons]
    window=compress
    densities=np.zeros((skeletons[t].shape[0]//compress,skeletons[t].shape[1]//compress),dtype=np.float)
    for xx in range(skeletons[t].shape[0]//compress):
        for yy in range(skeletons[t].shape[1]//compress):
            x = xx*compress
            y = yy*compress
            skeleton=skeletons[t][x-window:x+window,y-window:y+window]
            density = skeleton.count_nonzero()/((window*1.725)**2)
            densities[xx,yy]=density
    results = {}
    for kern_size in kern_sizes:
        density_filtered = gaussian_filter(densities,kern_size)
        sx = ndimage.sobel(density_filtered ,axis=0,mode='constant')
        sy = ndimage.sobel(density_filtered ,axis=1,mode='constant')
        sobel=np.hypot(sx,sy)
        results[kern_size] = density_filtered,sx,sy,sobel
    return(results)

In [2]:
path = directory_project
inst = 39,269,329
exp = get_exp(inst,path)
# exp.load_compressed_skel()

begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00


In [ ]:
kern_size=100
density_maps = [get_density_maps(exp,t,100,[10]) for t in range(exp.ts)]

In [16]:
density_maps[0][10][0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
kern_size=100

for index,density_map in enumerate(density_maps):
    plt.close('all')
    clear_output(wait=True)
    fig=plt.figure(figsize=(14,12))
    ax = fig.add_subplot(111)
    im= density_map[kern_size][0]
    figure = ax.imshow(im,vmax = 0.01)
    plt.colorbar(figure,orientation = 'horizontal')
    save = f'/home/cbisot/pycode/MscThesis/amftrack/notebooks/plotting/Figure/im*{index}.png'
    plt.savefig(save)


In [ ]:
img_array = []
for index in range(len(density_maps)):
    img = cv2.imread(f'/home/cbisot/pycode/MscThesis/amftrack/notebooks/plotting/Figure/im*{index}.png')
    img_array.append(img)
imageio.mimsave(f'/home/cbisot/pycode/MscThesis/amftrack/notebooks/plotting/Figure/movie_dense.gif', img_array,duration = 1)